## **แบบฝึกหัดขั้นสูง: Multi-Function Calling และการจัดการข้อมูลด้วย Pydantic**


---

### **ข้อที่ 1: ระบบวางแผนการเดินทางอัจฉริยะ**

**สถานการณ์:** ผู้ใช้ต้องการวางแผนการเดินทางจากโตเกียว (Tokyo) ไปยังปารีส (Paris) และต้องการข้อมูลเปรียบเทียบ ทั้งเรื่องเวลาท้องถิ่นของทั้งสองเมือง, สภาพอากาศปัจจุบัน, และเมืองหลวงของประเทศนั้นๆ เพื่อยืนยันข้อมูล

**งานของคุณ:**
1.  เขียนโค้ดที่รับคำสั่งจากผู้ใช้ เช่น `"ฉันจะเดินทางจากโตเกียวไปปารีส ช่วยบอกเวลาท้องถิ่นและสภาพอากาศของทั้งสองเมืองหน่อย แล้วเมืองหลวงของญี่ปุ่นกับฝรั่งเศสคือที่ไหนนะ?"`
2.  คุณจะต้องสร้าง Mock Function ใหม่ชื่อ `get_weather(city: str)` ซึ่งจะคืนค่าเป็นข้อมูลสภาพอากาศจำลอง (เช่น `{"temperature": "15°C", "condition": "Cloudy"}`)
3.  เรียกใช้ฟังก์ชัน `get_current_time`, `get_capital_city`, และ `get_weather` สำหรับเมืองทั้งสอง
4.  ** (ส่วนที่ท้าทาย)** ออกแบบ Pydantic Model ชื่อ `TripPlannerResponse` เพื่อรวบรวมข้อมูลทั้งหมด โดยมีการแบ่งโครงสร้างข้อมูลของแต่ละเมืองอย่างชัดเจน

**Pydantic Model ที่คาดหวัง:**
```python
from pydantic import BaseModel
from typing import Dict

class CityInfo(BaseModel):
    local_time: str
    weather: Dict[str, str] # e.g., {"temperature": "15°C", "condition": "Cloudy"}
    capital_of: str

class TripPlannerResponse(BaseModel):
    origin: CityInfo
    destination: CityInfo
    trip_summary: str # สร้างข้อความสรุปสั้นๆ จากข้อมูลทั้งหมด
```
**คำใบ้:** โมเดล LLM อาจจะต้องเรียกฟังก์ชันเดียวกัน (เช่น `get_current_time`) แต่ด้วยพารามิเตอร์ที่ต่างกัน (คนละเมือง) ในคำสั่งเดียว คุณต้องจัดการผลลัพธ์เหล่านี้ให้ถูกต้องและจับคู่กับเมืองที่ถูกต้องใน Pydantic Model

### **ข้อที่ 2: ระบบสรุปข้อมูลเหตุการณ์สำคัญ**

**สถานการณ์:** นักข่าวต้องการข้อมูลสรุปเกี่ยวกับเหตุการณ์ 'การประชุมสุดยอด AI' โดยต้องการทราบว่าจัดขึ้นที่ประเทศใด (สมมติว่าจัดขึ้นที่ฝรั่งเศส), เมืองหลวงของประเทศนั้น, เวลาท้องถิ่น ณ ตอนนั้น, และข่าวล่าสุดเกี่ยวกับ 'AI' และ 'เศรษฐศาสตร์ (economics)' เพื่อหาความเชื่อมโยง

**งานของคุณ:**
1.  เขียนโค้ดที่รับคำสั่งจากผู้ใช้ เช่น `"การประชุมสุดยอด AI จัดขึ้นที่ไหน ช่วยบอกเมืองหลวง เวลาปัจจุบัน และสรุปข่าวล่าสุดเกี่ยวกับ AI และเศรษฐศาสตร์ให้ที"`
2.  LLM ควร (infer) ได้ว่าต้องหาเมืองหลวงของฝรั่งเศส และเรียกใช้ `get_capital_city`, `get_current_time`, และ `get_latest_news` (สองครั้งสำหรับสองหัวข้อ)
3.  **(ส่วนที่ท้าทาย)** ออกแบบ Pydantic Model ที่สามารถรวบรวม 'หัวข้อข่าว' หลายๆ หัวข้อได้แบบไดนามิก และสร้างข้อความสรุปเชิงวิเคราะห์สั้นๆ จากข้อมูลทั้งหมด

**Pydantic Model ที่คาดหวัง:**
```python
from pydantic import BaseModel, Field
from typing import Dict

class EventBriefing(BaseModel):
    event_name: str = "AI Summit"
    location_country: str
    location_capital: str
    local_time: str
    news_briefings: Dict[str, str] = Field(..., description="A dictionary mapping news topics to their summaries")
    analytic_summary: str # ข้อความสรุปที่สร้างขึ้นเอง เช่น 'The AI Summit in Paris is happening amidst news of powerful AI models, which may impact the economy.'
```
**คำใบ้:** การที่ LLM ต้องเรียก `get_latest_news` สองครั้งสำหรับสองหัวข้อที่แตกต่างกัน เป็นการทดสอบความสามารถในการจัดการ Tool Calls ที่ซ้ำซ้อนแต่มีพารามิเตอร์ต่างกัน คุณต้องมี Logic ในการรวบรวมผลลัพธ์จาก Tool Calls เหล่านี้ลงใน Dictionary ของ Pydantic